In [1]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.system_object_type import FileSystemObjectType
import os
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )

In [2]:
def enum_folder(parent_folder, fn, ctx):
    """
    :type parent_folder: Folder
    :type fn: (File)-> None
    """
    parent_folder.expand(["Files", "Folders"]).get().execute_query()
    for file in parent_folder.files:  # type: File
        
        #download if it is a pdf
        url = file.properties['ServerRelativeUrl']
        if 'pdf' in url:
            fn(file , ctx)
            #downloadfile(url, ctx)
    for folder in parent_folder.folders:  # type: Folder
        enum_folder(folder, fn, ctx)


def print_file(f,ctx):
    """
    :type f: File
    """
    print(ctx.base_url+f.properties['ServerRelativeUrl'])

def print_props(f):
    for p in f.properties:
        print(p, f.properties[p])


In [3]:
def findDocs(target_folder_url, ctx):
    root_folder = ctx.web.get_folder_by_server_relative_path(target_folder_url)
    enum_folder(root_folder, print_file, ctx)

In [5]:
site_url = "https://hksinc.sharepoint.com/sites/BrainHealth"

ctx = ClientContext(site_url.strip()).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()
findDocs("Shared Documents", ctx)